<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Clustering using Vantage and Google Gemini
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;'><b>Introduction:</b></p>

<p style="font-size:16px;font-family:Arial;">This feature uses advanced clustering techniques powered by <b>Teradata Vantage</b> and <b>Google Gemini's Generative AI Embeddings</b> model to group similar customer complaints together. By identifying common themes and patterns, this functionality provides valuable insights into the key issues and pain points experienced by customers.</p>


<p style="font-size:16px;font-family:Arial;"><b>Key Features of Complaints Clustering:</b></p>
<ul style="font-size:16px;font-family:Arial;">
  <li>Leverages advanced clustering algorithms powered by <b>Teradata Vantage</b> and <b>Google Gemini's Generative AI Embeddings.</b></li>
  <li>Groups similar customer complaints together, revealing common themes and pain points.</li>
  <li>Provides clients with a deeper understanding of the key issues affecting their customers.</li>
  <li>Enables clients to prioritize and address the most pressing concerns more effectively.</li>
  <li>Helps clients identify opportunities for product improvements and enhanced customer experience.</li>
</ul>


<p style = 'font-size:16px;font-family:Arial;'>Unlock the revolutionary potential of Generative AI to categorize and analyze complaints with unparalleled efficiency.</p>

<p style = 'font-size:16px;font-family:Arial;'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Data Exploration</li>
     <li>Setup API key for Google Gemini</li>
    <li>Cluster the Complaints</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial;'>1. Configuring the environment</b>

<p style = 'font-size:18px;font-family:Arial;'><b>1.1 Install the required libraries</b></p>

In [ ]:
%%capture
!pip install -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style="height:2px;border:none;">
<p style = 'font-size:16px;font-family:Arial;'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import numpy as np
import timeit
import tqdm
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import *
tqdm_notebook.pandas()

# plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = "whitegrid")

# teradata lib
from teradataml import *
import teradataml
from teradataml import KMeans
from teradataml.analytics.valib import *
from teradataml import configure
from sqlalchemy import func
configure.val_install_location = "val"
configure.byom_install_location = "byom"


# genai
import getpass
import os
import google.generativeai as genai

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
display.max_rows = 5
display.max_rows = 5
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 5)

display.print_sqlmr_query = False
display.suppress_vantage_runtime_warnings = True
from IPython.display import display, Markdown

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>2. Connect to Vantage</b>
<p style = 'font-size:18px;font-family:Arial;'><b>2.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Complaints_Analysis_GenAI.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial;'><b>2.2 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>3. Data Exploration</b>

<p style = 'font-size:16px;font-family:Arial;'>Now, We will load the data and explore it for further steps/</p>

In [ ]:
df = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints'))

In [ ]:
df.shape

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>3.1 Graph for Count of Product Complaints Over Years</b></p>

<p style='font-size:16px;font-family:Arial;'>The provided graph visualizes the count of complaints over the past few years, categorized by product names.</p>

In [ ]:
viz_df = df.assign(year = func.td_year_of_calendar(df.date_received.expression))

In [ ]:
pd_df = viz_df.select(['product','year','complaint_id']).groupby(['product', 'year']).agg(['count']).to_pandas()

In [ ]:
# Sorting the DataFrame by year for each product
pd_df_sorted = pd_df.sort_values(by=['product', 'year'])

# Plotting using Plotly
fig = px.line(pd_df_sorted, x='year', y='count_complaint_id', color='product', markers=True, title='Count of Product Complaints Over Years')
fig.update_layout(xaxis_title='Year', yaxis_title='Count', legend_title='Product', width=1200, height=600)

fig.show(renderer="notebook")

<hr style='height:1px;border:none;'> 
<p style='font-size:18px;font-family:Arial;'><b>3.2 Graph for Count of Complaints by Months</b></p> 
<p style='font-size:16px;font-family:Arial;'>The provided graph visualizes the count of complaints by months. We can see that the mean count is above 500, and the July and August months have the maximum complaints count.</p>

In [ ]:
df = df.assign(complaint_month = func.td_month_of_year(df.date_received.expression))
grp_gen = df.select(['complaint_month','complaint_id']).groupby(['complaint_month']).agg(['count']).to_pandas()

# Define a reverse mapping dictionary
reverse_month_mapping = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
                         7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}

# Create a new column with month names based on reverse mapping
grp_gen['month'] = grp_gen['complaint_month'].map(reverse_month_mapping)


fig = px.bar(grp_gen.sort_values(by = 'complaint_month'), x='month', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'month': 'Complaint Month'},
             title='Number of Complaints by Month')

# Add hover information
fig.update_traces(hovertemplate='Month: %{x}<br>Number of Complaints: %{y:,}')

fig.show(renderer="notebook")

<hr style='height:1px;border:none;'> 

<p style='font-size:18px;font-family:Arial;'><b>3.3 Graph for Number of Complaints by Product</b></p> <p style='font-size:16px;font-family:Arial;'>The graph displays the number of complaints received for different products. The data shows that the highest number of complaints are related to credit cards or prepaid cards, as well as credit reporting and credit repair services.</p>

In [ ]:
# Assuming df is your DataFrame and sns is seaborn

grp_gen = df.select(['product','complaint_id']).groupby(['product']).agg(['count']).to_pandas()

fig = px.bar(grp_gen, x='product', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'product': 'Product'},
             title='Number of Complaints by Product')

# Add hover information
fig.update_traces(hovertemplate='Product: %{x}<br>Number of Complaints: %{y:,}')

fig.show(renderer="notebook")

<hr style='height:1px;border:none;'> 

<p style='font-size:18px;font-family:Arial;'><b>3.4 Graph for Number of Complaints by Issue</b></p> <p style='font-size:16px;font-family:Arial;'>The graph displays the number of complaints received for different issues. The data shows that the highest number of complaints are related to issue of incorrect information on your report.</p>

In [ ]:
# Assuming df is your DataFrame and sns is seaborn

grp_gen = df.select(['issue','complaint_id']).groupby(['issue']).agg(['count']).to_pandas()

grp_gen = grp_gen.sort_values('count_complaint_id', ascending = False)[:10]

fig = px.bar(grp_gen, x='issue', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'issue': 'Issue'},
             title='Number of Complaints by Issue(Top 10)')

# Add hover information
fig.update_traces(hovertemplate='Issue: %{x}<br>Number of Complaints: %{y:,}')

fig.show(renderer="notebook")

<hr style='height:1px;border:none;'> 

<p style='font-size:18px;font-family:Arial;'><b>3.5 Graph for Number of Complaints by Sub-Issue</b></p> 

<p style='font-size:16px;font-family:Arial;'>The graph displays the number of complaints received for different sub-issues. The data shows that the highest number of complaints are related to issue of information belongs to someone else.</p>

In [ ]:
# Assuming df is your DataFrame and sns is seaborn

grp_gen = df.select(['sub_issue','complaint_id']).groupby(['sub_issue']).agg(['count']).to_pandas()

grp_gen = grp_gen.sort_values('count_complaint_id', ascending = False)[:10]

fig = px.bar(grp_gen, x='sub_issue', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'sub_issue': 'Sub-Issue'},
             title='Number of Complaints by Sub-Issue(Top 10)')

# Add hover information
fig.update_traces(hovertemplate='Sub-Issue: %{x}<br>Number of Complaints: %{y:,}')

fig.show(renderer="notebook")

<hr style='height:1px;border:none;'> 

<p style='font-size:18px;font-family:Arial;'><b>3.6 Graph for Number of Complaints by Channel</b></p>

<p style='font-size:16px;font-family:Arial;'>The graph displays the number of complaints received for different issues. The data shows that the all the complaints are submitted by web channel.</p>

In [ ]:
grp_gen = df.select(['submitted_via','complaint_id']).groupby(['submitted_via']).agg(['count']).to_pandas()

# Create a mapping of numbers to product names
product_mapping = {i: product for i, product in enumerate(grp_gen['submitted_via'])}

# Replace product names with numbers in the DataFrame
grp_gen['product_num'] = grp_gen['submitted_via'].map({product: i for i, product in enumerate(grp_gen['submitted_via'])})

fig = px.bar(grp_gen, x='submitted_via', y='count_complaint_id',
             labels={'count_complaint_id': 'Number of Complaints', 'submitted_via': 'Submitted Via'},
             title='Number of Complaints by Channel')

# Add hover information
fig.update_traces(hovertemplate='Submitted Via: %{x}<br>Number of Complaints: %{y:,}')

fig.show(renderer="notebook")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>4. Setup API key for Google Gemini</b>
<p style = 'font-size:16px;font-family:Arial;'>Please enter the Google API Key, if you don't have one, please get it from <a href = 'https://ai.google.dev/gemini-api/docs/api-key'>here</a></p>

In [ ]:
GOOGLE_AI_API_KEY = getpass.getpass(prompt = 'Please enter GOOGLE_AI_API_KEY: ')
genai.configure(api_key = GOOGLE_AI_API_KEY)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>5. Cluster the Complaints</b>

<p style = 'font-size:16px;font-family:Arial;'>For our complaint clustering task, we'll be using a sample of the data to cluster the complaints. This approach will allow us to effectively analyze and categorize the complaints without using the entire dataset.</p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial;'><b>5.1 Do you want to generate the embeddings?</b></p>    
<p style = 'font-size:16px;font-family:Arial;'>We have already generated embeddings for the comment text and stored them in <b>Vantage</b> table.</p>

<center><img src="images/decision_emb_gen_cmt.svg" style="border: 4px solid #404040; border-radius: 10px;" alt="embeddings_decision" width=300 height=400/></center>

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial;'><i><b>Note: If you would like to skip the embedding generation step to save the time and move quickly to next step, please enter "No" in the next prompt.</b></i></p>
</div>

<p style = 'font-size:16px;font-family:Arial;'>To save time, you can move to the already generated embeddings section. However, if you would like to see how we generate the embeddings, or if you need to generate the embeddings for a different dataset, then continue to the following section.  
<br>
<br>
For demonstration purposes, the embedding generation step is currently limited to 10 sample rows using:<code> embedding_df = create_embeddings(df.to_pandas(<b>num_rows=10</b>))</code>.
If you want to generate embeddings for the entire dataset, simply remove the <code>num_rows=10</code> argument.
</p>

In [ ]:
# Request user's input
generate = input("Do you want to generate embeddings? ('yes'/'no'): ")

# Check the user's input
if generate.lower() == 'yes':
    print("\nGreat! We'll start by generating embeddings.")

    print("\nGenerating embeddings, please wait...")
    
    display(Markdown("""<br><div class="alert alert-block alert-warning">
            <p style = 'font-size:16px;font-family:Arial;'><i><b>Note: The embedding generation step is estimated to take approximately 5 minutes to complete. Please be patient.</b></i></p>
        </div>"""))
    
    display(Markdown("""<p style = 'font-size:16px;font-family:Arial;'>Here we have used <b>text-embedding-004</b> model google gemini offerings. 
please click here to learn more about this embeddings model <a href="https://ai.google.dev/gemini-api/docs/embeddings">embeddings</a></p>"""))
    
    from google.api_core import retry
    def make_embed_text_fn(model):

        @retry.Retry(timeout=300.0)
        def embed_fn(text: str) -> list[float]:
            # Set the task_type to CLUSTERING.
            embedding = genai.embed_content(model=model,
                                            content=text,
                                            task_type="clustering")
            return embedding["embedding"]

        return embed_fn

    def create_embeddings(df):
        model = 'models/text-embedding-004'
        df['Embeddings'] = df['consumer_complaint_narrative'].progress_apply(make_embed_text_fn(model))

        # Generate all the embeddings columns from the "embeddings" column.
        for i in range(len(df['Embeddings'][0])):
            df["embeddings_{}".format(i)] = df["Embeddings"].apply(lambda x: x[i])

        # drop embedding 
        df = df.drop("Embeddings", axis=1)
        return df

    # API request for embeddings for 10 sample rows.
    embedding_df = create_embeddings(df.to_pandas(num_rows=10))

    # Save embeddings to SQL
    print("\nSaving embeddings to the database...")
    copy_to_sql(
        df = embedding_df,
        table_name = 'complaints_embeddings',
        if_exists = 'replace'
    )

    print("\nEmbeddings generated and saved successfully!")

elif generate.lower() == 'no':
    print("\nLoading embeddings from the local file 'embeddings.parquet.gzip'.")
    
    display(Markdown("""<br><p style = 'font-size:16px;font-family:Arial;'>The code above first reads the data from the files. The files contain information about the complaints embeddings. The code then loads the data into a permanent table in SQL. Once the data is loaded, we will use the Vantage in-database function <code>KMeans</code> to find the clusters from the embeddings. The data contains complaints embeddings, which are lists of numerical values, or vectors.</p>
    <p style = 'font-size:16px;font-family:Arial;'>The embeddings file contains 1000 records, each with 1536 numerical features. This means that the file is quite large and it may take some time to load it into SQL.</p>
    <div class="alert alert-block alert-info" id="no-azure">
    <p style = 'font-size:16px;font-family:Arial;'><i><b>Note</b>: Please be patient. The code above is loading data from files and copying it to SQL. This process may take 10-15 seconds.</i></p>
</div>"""))
    # Save them to SQL
    copy_to_sql(
        df = pd.read_parquet('./data/embeddings.parquet.gzip'),
        table_name = 'complaints_embeddings',
        if_exists = 'replace'
    )

    print("\nEmbeddings loaded and saved successfully!")

else:
    print("\nInvalid input. Please enter 'yes' or 'no' to proceed.")

In [ ]:
data_cols = DataFrame('complaints_embeddings').iloc[:, 17].columns + DataFrame('complaints_embeddings').iloc[:, 19:].columns
KMeans_Model = KMeans(
    data = DataFrame('complaints_embeddings')[data_cols],
    id_column = "complaint_id",
    target_columns = DataFrame('complaints_embeddings').columns[21:],
    output_cluster_assignment = True,
    num_clusters = 5
)

In [ ]:
print("Data information: \n", KMeans_Model.model_data.shape)

In [ ]:
KMeans_Model.result

In [ ]:
embeddings_cluster = DataFrame('complaints_embeddings').join(
    other = KMeans_Model.result,
    how = "inner",
    on = "complaint_id=complaint_id",
    lprefix =  "L_"
)

In [ ]:
# View complaints in cluster 1
embeddings_cluster[['td_clusterid_kmeans','complaint_id','consumer_complaint_narrative']] \
                    .loc[embeddings_cluster.td_clusterid_kmeans == 1]

<hr style='height:1px;border:none;'> 

<p style='font-size:18px;font-family:Arial;'><b>5.1 Visualization of Clusters with Complaints</b></p> 

<p style='font-size:16px;font-family:Arial;'>The graph displays the clustering of complaints into distinct groups. Based on the analysis, the data has been divided into 5 optimal clusters, each representing a unique pattern or category of complaints. This clustering approach helps to identify the key areas or types of complaints that are most prevalent, allowing for more targeted investigation and resolution efforts.</p>

In [ ]:
# emb = DataFrame('kmeans_features').to_pandas()
clus = embeddings_cluster.to_pandas()

In [ ]:
clus

In [ ]:
from sklearn.manifold import TSNE

#Note: For smaller datasets (e.g., 10 sample records), set perplexity=3. Example: tsne = TSNE(n_components=2, random_state=42, perplexity=3)
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(clus.iloc[:, 20:-1])

In [ ]:
tsne_df = pd.DataFrame(tsne_result, columns=['tsne_1', 'tsne_2'])
tsne_df['cluster_id'] = clus['td_clusterid_kmeans']
tsne_df['complaint_id'] = clus['consumer_complaint_narrative']

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming you have already computed tsne_df as per the previous example

# Create a new DataFrame combining t-SNE results with complaint information
tsne_complaint_df = pd.DataFrame(tsne_result, columns=['tsne_1', 'tsne_2'])
tsne_complaint_df['cluster_id'] = clus['td_clusterid_kmeans']
tsne_complaint_df['complaint_id'] = clus['complaint_id']
tsne_complaint_df['complaint'] = clus['consumer_complaint_narrative']

# Truncate text for hover data
max_chars = 50  # Maximum characters to display
tsne_complaint_df['truncted_complaint'] = clus['consumer_complaint_narrative'].apply(lambda x: x[:max_chars] + '...' if len(x) > max_chars else x)

# Plot using Plotly Express
fig = px.scatter(tsne_complaint_df, x='tsne_1', y='tsne_2', color='cluster_id',
                 hover_data=['complaint_id', 'truncted_complaint', 'cluster_id'])

fig.update_traces(marker=dict(size=15))
fig.update_layout(
    title='t-SNE Visualization of Clusters with Complaints',
    xaxis_title='dimension-1',
    yaxis_title='dimension-2',
    xaxis=dict(tickangle=45),
    width=1000,
    height=800,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"
    ),
    autosize=False,
)

# Customize the hovertemplate
fig.update_traces(hovertemplate="<b>Complaint ID:</b> %{customdata[0]}<br>"
                                 "<b>Complaint:</b> %{customdata[1]}<br>"
                                 "<b>Cluster ID:</b> %{customdata[2]}<br><extra></extra>")

fig.show(renderer="notebook")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>6. Cleanup</b>

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial;'><b>6.1 Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
for table in db_list_tables()['TableName'].tolist():
    try:
        db_drop_table(table_name=table, schema_name="demo_user")
    except:
        pass

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial;'> <b>6.2 Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>